## KFServing Example 

In [4]:
#Use subprocess to upgrade the kfserving python package via the Linux terminal

import subprocess
data = subprocess.Popen(['pip', 'install','--upgrade', 'kfserving'], stdout = subprocess.PIPE)

In [16]:
#Import gcsfs to access any files from the GCS bucket

import gcsfs
fs = gcsfs.GCSFileSystem(project='YDataSynthetic', token = 'cloud')

In [10]:
#The Kfserving python sdk interfaces with the Kubernetes cluster to create the 

import kfserving
from kubernetes import client
from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TransformerSpec
from kfserving import V1alpha2SKLearnSpec, V1alpha2PyTorchSpec
from kfserving import V1alpha2CustomSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1Container
from kubernetes.client import V1ResourceRequirements
import kubernetes.client
import os
import requests 
import json
import numpy as np

In [11]:
namespace = utils.get_default_target_namespace()

api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
print(api_version, namespace, constants.KFSERVING_KIND)

#enter the specifications for your kfserving model and endpoints
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            min_replicas=1,
                            sklearn=V1alpha2SKLearnSpec(

                                #Store your model with the name model.joblib and provide the file location 
                                # to the storage_uri variable
                                
                              storage_uri='gs://pipelines_artifacts/kfserving/sklearn/randomforest',
                              resources=V1ResourceRequirements(
                                  
                                  #This model requires higher cpu and memory allocation than usual
                                  
                                  requests={'cpu':'500m','memory':'4Gi'},
                                  limits={'cpu':'500m', 'memory':'4Gi'}))))

#Go to Chrome Settings -> Cookies and Other site Data -> See all Cookies -> <Your YData Instance link>
#Copy the key in the 'content' under "authservice_session" cookie into the 'session' variable

session = "MTYwNzI1MDgxNXxOd3dBTkZneVJrdEVORVJhVGpZMFZUVTBVVlpSVDFCRFRsQlFWMHN5VEZZM1NEZEJORlZVVVVkWlZVcEtVa2N5V2t0VFRsVlFXVUU9fMRsPNyuFCYAukl02nmr8bJj5IJvY2_MtyZBMTNucv33"
headers = { 'Host': 'randomforest.' + namespace + '.example.com', 'Cookie': 'authservice_session=' + session }

isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='randomforest', namespace=namespace),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))

KFServing = KFServingClient()
KFServing.create(isvc)
KFServing.get('randomforest', namespace=namespace, watch=True, timeout_seconds=120)

serving.kubeflow.org/v1alpha2 ruju-demo-work InferenceService
NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
randomforest         Unknown                                                                                      
randomforest         False                                                                                        
randomforest         False                                                                                        
randomforest         False                                                                                        
randomforest         False                                                                                        
randomforest         True       100                             http://randomforest.ruju-demo-work.example.com    


In [13]:
KFServing.get('randomforest', namespace=namespace, watch=True, timeout_seconds=120)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
randomforest         True                   100                 http://randomforest.ruju-demo-work.example.com    


In [19]:
session = "MTYwNzMzNzI1NHxOd3dBTkRSQlJVWkVSVmhhUVZnMVNEWk1TVFpDUTFwRlVWUXpRVkJFUXpJMVR6UkdORXhWV2tOSFV6ZFNORk0xUVZsSFNESTFUVUU9fDYNLTvscaPyK_tkOYWKTMKrLFU2AaU57-bqvL3AuVeI"
url = "https://ruju.dev.google.ydata.ai/v1/models/randomforest:predict"
headers = { 'Host': 'randomforest.' + namespace + '.example.com', 'Cookie': 'authservice_session=' + session }
print(url)
print("\n")
print(headers)

https://ruju.dev.google.ydata.ai/v1/models/randomforest:predict


{'Host': 'randomforest.ruju-demo-work.example.com', 'Cookie': 'authservice_session=MTYwNzMzNzI1NHxOd3dBTkRSQlJVWkVSVmhhUVZnMVNEWk1TVFpDUTFwRlVWUXpRVkJFUXpJMVR6UkdORXhWV2tOSFV6ZFNORk0xUVZsSFNESTFUVUU9fDYNLTvscaPyK_tkOYWKTMKrLFU2AaU57-bqvL3AuVeI'}


In [18]:
with open('./input.json') as json_file:
    data = json.load(json_file)
    print(url, headers)
    response = requests.post(url, json.dumps(data), headers=headers)
    print("\n")
    print(json.dumps(data))
    print("\n")
    print(response)
    print("---\n")
    probs = json.loads(response.content.decode('utf-8'))
    print(probs)
    print("Predicted Churn Flag: {}".format(np.argmax(probs)))

https://ruju.dev.google.ydata.ai/v1/models/randomforest:predict {'Host': 'randomforest.ruju-demo-work.example.com', 'Cookie': 'authservice_session=MTYwNzMzNzI1NHxOd3dBTkRSQlJVWkVSVmhhUVZnMVNEWk1TVFpDUTFwRlVWUXpRVkJFUXpJMVR6UkdORXhWV2tOSFV6ZFNORk0xUVZsSFNESTFUVUU9fDYNLTvscaPyK_tkOYWKTMKrLFU2AaU57-bqvL3AuVeI'}


{"instances": [[0.0, 1.0, 0.0, 0.0, 0.0, 49.0, 1.0, 1.0, 103.7, 5036.3, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]]}


<Response [200]>
---

{'predictions': [0]}
Predicted Churn Flag: 0


In [20]:
#Finally, it is a good practice to delete the inferenceservice, which clears up all the pods, 
#services and other resources used up by it

KFServing.delete('randomforest', namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'randomforest',
  'group': 'serving.kubeflow.org',
  'kind': 'inferenceservices',
  'uid': '7309da6a-6762-4579-ab6e-634239c283cb'}}